In [5]:
from textblob import TextBlob
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


from warnings import filterwarnings
filterwarnings('ignore')

In [6]:
import pandas as pd 
data = pd.read_csv("train.tsv", sep = "\t")

## binary sınıflandırma

In [7]:
data.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [8]:
data["Sentiment"].replace(0, value = "negatif", inplace = True)
data["Sentiment"].replace(1, value = "negatif", inplace = True)

In [9]:
data["Sentiment"].replace(3, value = "pozitif", inplace = True)
data["Sentiment"].replace(4, value = "pozitif", inplace = True)

In [10]:
data = data[(data.Sentiment == "negatif") | (data.Sentiment == "pozitif")]

In [11]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,negatif
21,22,1,good for the goose,pozitif
22,23,1,good,pozitif
33,34,1,"the gander , some of which occasionally amuses...",negatif
46,47,1,amuses,pozitif


In [12]:
data.groupby("Sentiment").count()

,PhraseId,SentenceId,Phrase
Sentiment,,,
negatif,34345,34345,34345
pozitif,42133,42133,42133


In [13]:
df = pd.DataFrame()
df["text"] = data["Phrase"]
df["label"] = data["Sentiment"]
df.head()

,text,label
0,A series of escapades demonstrating the adage ...,negatif
21,good for the goose,pozitif
22,good,pozitif
33,"the gander , some of which occasionally amuses...",negatif
46,amuses,pozitif


In [14]:
#Veri Ön İşleme
#buyuk-kucuk
df["text"] = df["text"].apply(lambda x: " ".join(x.lower() for x in x.split()))
#noktalama işaretleri
df["text"] = df["text"].str.replace('[^\w\s]', '')
#sayılar
df["text"] = df["text"].str.replace('\d', '')
#stopwords
import nltk
from nltk.corpus import stopwords
sw = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
#seyreklerin silinmesi
sil = pd.Series(' '.join(df['text']).split()).value_counts()[-1000:]
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))
#lemmi
from textblob import Word
#nltk.download('wordnet')
df['text'] = df['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 

In [15]:
#Değişken Mühendisliği
#count vectors:
#tf-ıdf vectors(words,characters,n-grams)
#word embeddings

#tf(t) = (bir t teriminin bir dökümanda gözlenme frekansı) / (dökümandaki toplam terim sayısı)
#idf(t) = log_e(toplam döküman sayısı/içinde t terimi olan belge sayısı)

In [16]:
#tf-ıdf vectors:durum bize tf idf terimlerin dökümanlar ve tüm korpus göz önünde bulundurularak göreceli etkilerini ifade eder

In [17]:
#word embeddings:

In [18]:
#(bir t teriminin bir dökümanda gözlenme frekansı):mesela ber text içindeki good sayısı
#(dökümandaki toplam terim sayısı):bu text içindeki kelime sayısı
#(toplam döküman sayısı):bu set içindeki text veya döküman sayısı
#(içinde t terimi olan belge sayısı):kaç tane belgede t vardır 

In [19]:
#her bir satır bir yorumu veya dökümanı temsil eder
#her bir sutunda her bir dökümandaki kelimeleri ifade eder
#kesişimlerindeki ifadelere ise dökümanlarda geçme sıklığını ifade eder

In [20]:
df.head()

,text,label
0,series demonstrating adage good goose also goo...,negatif
21,good goose,pozitif
22,good,pozitif
33,gander occasionally amuses none amount much story,negatif
46,amuses,pozitif


In [21]:
df.iloc[0]

text     series demonstrating adage good goose also goo...
label                                              negatif
Name: 0, dtype: object

In [22]:
#glove, fasttext, word2vec sentiment analizleri için öğretilmiş ağlar(modeller)

In [23]:
#test-train

In [24]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df["text"],
                                                                   df["label"])

In [25]:
train_y

102757    pozitif
40372     pozitif
50261     pozitif
124242    negatif
128147    negatif
           ...   
48745     pozitif
85099     pozitif
950       pozitif
146626    pozitif
94624     negatif
Name: label, Length: 57358, dtype: object

In [26]:
test_y

54828     pozitif
95150     pozitif
62776     pozitif
22526     negatif
68879     negatif
           ...   
121838    negatif
77093     pozitif
103353    pozitif
104178    negatif
9794      negatif
Name: label, Length: 19120, dtype: object

In [27]:
#bağımlı değişkene dönüştürme işlemi
encoder = preprocessing.LabelEncoder()

In [28]:
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)#0 1 haline çevirdik

In [29]:
train_y

array([1, 1, 1, ..., 1, 1, 0])

In [30]:
test_y

array([1, 1, 1, ..., 1, 0, 0])

In [31]:
#Count Vectors

In [32]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

CountVectorizer()

In [33]:
x_train_count = vectorizer.transform(train_x)
x_test_count = vectorizer.transform(test_x)

In [34]:
vectorizer.get_feature_names()[0:5]

['aaliyah', 'abagnale', 'abandon', 'abandoned', 'abbass']

In [35]:
x_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [36]:
#TF-IDF

In [37]:
#tf idf ile word seviye özelliklerimizi oluşturduk

In [38]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x)

TfidfVectorizer()

In [39]:
x_train_tfidfword = tf_idf_word_vectorizer.transform(train_x)
x_test_tfidfword = tf_idf_word_vectorizer.transform(test_x)

In [40]:
tf_idf_word_vectorizer.get_feature_names()[0:5]

['aaliyah', 'abagnale', 'abandon', 'abandoned', 'abbass']

In [41]:
#ngram level tf idf

In [42]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range = (2,3))
tf_idf_ngram_vectorizer.fit(train_x)

TfidfVectorizer(ngram_range=(2, 3))

In [43]:
x_train_tfidfngram = tf_idf_ngram_vectorizer.transform(train_x)
x_test_tfidfngram = tf_idf_ngram_vectorizer.transform(test_x)

In [44]:
tf_idf_ngram_vectorizer.get_feature_names()[0:5]

['aaliyah one',
 'aaliyah one starring',
 'abagnale antic',
 'abandon hold',
 'abandon hold promise']

In [45]:
#characters level tf-idf

In [46]:
tf_idf_chars_vectorizer = TfidfVectorizer(analyzer = "char", ngram_range = (2,3))
tf_idf_chars_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

In [47]:
x_train_tfidfchars = tf_idf_chars_vectorizer.transform(train_x)
x_test_tfidfchars = tf_idf_chars_vectorizer.transform(test_x)

# Makine Öğrenmesi ile Sentiment Sınıflandırması

# Lojistik Regresyon

In [48]:
x_train_count

<57358x13546 sparse matrix of type '<class 'numpy.int64'>'
	with 281897 stored elements in Compressed Sparse Row format>

In [49]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_count,
                                           test_y,
                                           cv = 10).mean()
print("Count Vectors Doğruluk Oranı:", accuracy)# %83lük bir doğruluk oranı

Count Vectors Doğruluk Oranı: 0.8368200836820083


In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tfidfword, train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tfidfword,
                                           test_y,
                                           cv = 10).mean()
print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)# %82lik bir doğruluk oranı

In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tfidfngram, train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tfidfngram,
                                           test_y,
                                           cv = 10).mean()
print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)# %74lük bir doğruluk oranı

In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tfidfchars, train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tfidfchars,
                                           test_y,
                                           cv = 10).mean()
print("CHARLEVEL Doğruluk Oranı:", accuracy)# %78lik bir doğruluk oranı

# Naive Bayes

In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)# %83lik bir doğruluk oranı

In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tfidfword,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tfidfword, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)# %83lik bir doğruluk oranı

In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tfidfngram,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tfidfngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)# %76lik bir doğruluk oranı

In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tfidfchars,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           x_test_tfidfchars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)# %74lik bir doğruluk oranı

# Random Forest

In [ ]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)# %82lik bir doğruluk oranı

In [ ]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tfidfword,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_tfidfword, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)# %82lik bir doğruluk oranı

In [ ]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tfidfngram,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_tfidfngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)# %lik bir doğruluk oranı

In [ ]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tfidfchars,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_tfidfchars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)# %lik bir doğruluk oranı

# XGBoost

In [ ]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)# %82lik bir doğruluk oranı

In [ ]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tfidfword,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           x_test_tfidfword, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)# %82lik bir doğruluk oranı

In [ ]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tfidfngram,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           x_test_tfidfngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)# %82lik bir doğruluk oranı

In [ ]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tfidfchars,train_y)
accuracy = model_selection.cross_val_score(rf_model, 
                                           x_test_tfidfchars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)# %78lik bir doğruluk oranı

In [ ]:
#en etkiliyi alalım

In [ ]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_count,
                                           test_y,
                                           cv = 10).mean()
print("Count Vectors Doğruluk Oranı:", accuracy)# %83lük bir doğruluk oranı

In [50]:
a = pd.Series("yes i like this film")
b = pd.Series("very bad")
c = pd.Series("no not good look at that shit very bad")
v = CountVectorizer()
v.fit(train_x)

CountVectorizer()

In [51]:
a = v.transform(a)
b = v.transform(b)
c = v.transform(c)

In [55]:
a.toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [52]:
loj_model.predict(a)

array([1])

In [53]:
loj_model.predict(b)

array([0])

In [54]:
loj_model.predict(c)

array([1])